In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.stats import qmc
from scipy.stats import norm
import itertools


In [3]:
def get_bigger_power2(n):
    k = 0
    m = 0
    while n > k:
        m += 1
        k = 2**m
    return m

# Low Discrepancy sequences

Sobol sequence

In [4]:
def Sobol_seq_normal_transformed(d,m, number_paths, scramble_func, number_of_run):
    Sobol_seq = qmc.Sobol(d, scramble = scramble_func) # weil scramble ist noch irgendein Extraschritt und default ist True

    n = get_bigger_power2(number_of_run * number_paths)

    if scramble_func == True:
        sample =  Sobol_seq.random_base2(n)
        sample = sample[(number_of_run-1) * number_paths:number_of_run * number_paths,:]
        sample = np.where(sample == 0, np.random.uniform(), sample)  #DAS HIER IST NUR HIER GEÄNDERT
        sample = np.where(sample == 0, 0.5, sample)
        sample = np.where(sample == 1, 0.5, sample)
    elif scramble_func == False:
        sample = Sobol_seq.random_base2(n+1) #aufgrund der Konstruktion berechnen wir 2 mal so viele samples, trennen die erste Zeile ab und nehmen dann die richtige Anzahl
        sample = sample[1:,:] #erste Stelle wird weggeschnitten, weil niht brauchbar

        sample = sample[(number_of_run-1) * number_paths:number_of_run * number_paths,:]
        sample = np.where(sample == 0, np.random.uniform(), sample)  #DAS HIER IST NUR HIER GEÄNDERT
        sample = np.where(sample == 0, 0.5, sample)
        sample = np.where(sample == 1, 0.5, sample)


    #hier nachher wieder ändern
    #sample[0,:] = np.zeros(d)
    #print(sample)

    #uniform random variable to normal distributed random varianle
    return norm.ppf(sample,loc = 0, scale = 1) #scale is standard deviation not variance!!!

Halton sequence

In [5]:
def Halton_seq_normal_transformed(d, mc_runs, scramble_func, number_of_run):
    Halton_seq = qmc.Halton(d, scramble=scramble_func)
    if scramble_func == False:
        sample = Halton_seq.random(n=number_of_run * mc_runs+1)
        sample=sample[1:,:]                      #ohne sampling nehmen wir die erste Zeile raus, da dorrt nur Nullen stehen, die man nicht sinnvoll invertieren kann
        sample = sample[(number_of_run-1)*mc_runs:,:] #wir wollen immer nur eine Länge von mc_runs aber non-overlapping
    elif scramble_func == True:
        sample = Halton_seq.random(n=number_of_run * mc_runs)
        sample = sample[(number_of_run-1)*mc_runs:,:] #wir wollen immer nur eine Länge von mc_runs aber non-overlapping
    #display(sample)
    #display(norm.ppf(sample, loc = 0, scale = 1))
    return norm.ppf(sample, loc = 0, scale = 1)

In [6]:
def get_BB_power2(n):
    k = 0
    while n%2**k == 0:
        k=k+1
    return (k-1)

BB Konstruktion

In [7]:
def get_indices_to_calculate(X):
    to_calculate = np.argwhere(np.isnan(X[0,:]) == True)
    return to_calculate

def get_indices_is_calculated(X):
    is_calculated = np.argwhere(np.isnan(X[0,:]) == False)
    return is_calculated

def create_BB(number_paths, M, sigma, g, delta_t, m, scramble_func, Sobol_not_Halton, QMC, norm_rv):
    h = M                       #h : Differenz der Punkte in Indexpunkten   Sit das h hier das a aus dem paper?
    X = np.empty((number_paths,M+1))       #+1 damit die 0 am Anfang mit dabei ist
    X[:] = np.nan
    X[:,0] = 0

    if QMC == False:
        norm_rv = norm_rv

    elif QMC == True:
        if Sobol_not_Halton == True:
            norm_rv = norm_rv
        elif Sobol_not_Halton == False:
            norm_rv = norm_rv
    
    current_pos_rv = 0

    X[:,M] = sigma * np.sqrt(M * delta_t) * norm_rv[:,current_pos_rv] #M-1 damit man letzten Eintrag M trifft
    current_pos_rv += 1

    for i in range(g):
        h = int(h/2)
        pos = h
        for j in range(2**i):
            X[:,pos] = (1/2) * X[:,pos - h] + (1/2) * X[:,pos + h] + sigma * np.sqrt((1/2) * h * delta_t) * norm_rv[:, current_pos_rv]
            #print(pos)
            pos = pos + 2*h
            current_pos_rv +=1

    #jetzt die restlichen Werte ausfüllen
    #wir betrachten nur die erste Zeile, weil in den weiteren Pfaden an den gleichen Stellen die Werte fehlen

    i = np.where(np.isnan(X[0,:]) == False)[0][0] #Startwert ist erster Eintrag, der nicht nan ist
    while np.isnan(X).any():
        to_calculate = get_indices_to_calculate(X)
        j = 0
        while to_calculate[j] <= i:
            j +=1

        i = to_calculate[j][0]


        #nun suchen wir den k Index, also den oberen Wert für die Berechnung
        o = 0
        is_calculated = get_indices_is_calculated(X)
        while is_calculated[o] <= i:    #Achtung neues i, nicht das von unten!
            o+=1
        k = is_calculated[o][0]

        j = is_calculated[o-1][0]

        X[:,i] = ((k-i)/(k-j)) * X[:,j] + ((i-j)/(k-j)) * X[:,k] + sigma * np.sqrt(((k-i)/(k-j)) * delta_t) * norm_rv[:,current_pos_rv]
        current_pos_rv += 1
    return X

Standard Brownian Motion berechnen

In [8]:
def create_standard_bm(sigma, number_paths, M, delta_t, m, scramble_func, Sobol_not_Halton, QMC, norm_rv):
    w = np.empty((number_paths,M))

    if QMC == False:
        for m in range(number_paths):
            #create lower trinangular matrix with ones
            lower_tri_matr = np.tril(np.ones([M,M]))
            A =  math.sqrt(delta_t)*lower_tri_matr #! hier sigma multipiziert, ist das so richtig?
            normal_sequence = norm_rv[m,:]
            #create Brownian motion
            w[m,:] = sigma * A.dot(normal_sequence)

    elif QMC == True:
        if Sobol_not_Halton == True:
            Sobol_sequence = norm_rv
            for l in range(number_paths):
                #create lower trinangular matrix with ones
                lower_tri_matr = np.tril(np.ones([M,M]))
                A = sigma * math.sqrt(delta_t)*lower_tri_matr #! hier sigma multipiziert, ist das so richtig?
                #create Brownian motion
                w[l,:] = A.dot(Sobol_sequence[l,:])
        elif Sobol_not_Halton == False: 
            Halton_sequence = norm_rv
            for l in range(number_paths):
                #create lower trinangular matrix with ones
                lower_tri_matr = np.tril(np.ones([M,M]))
                A = sigma * math.sqrt(delta_t)*lower_tri_matr #! hier sigma multipiziert, ist das so richtig?
                #create Brownian motion
                w[l,:] = A.dot(Halton_sequence[l,:])

        
    w = np.concatenate((np.reshape(np.zeros(number_paths),(number_paths,1)), w),axis=1)
    return w

Berechnung von i_K, w_k, u_k, c_k, r_k, m_k

In [9]:
def create_i_k(BB, number_paths, M, sigma, g, delta_t, m, scramble_func, Sobol_not_Halton, QMC, i_0, K_0, norm_rv):
    if BB == True:
        X = create_BB(number_paths, M, sigma, g, delta_t, m, scramble_func, Sobol_not_Halton, QMC, norm_rv)
    elif BB == False:
        X = create_standard_bm(sigma, number_paths, M, delta_t, m, scramble_func, Sobol_not_Halton, QMC, norm_rv)
    #display(X)    
    #create differences of the Wiener process
    delta_X = np.diff(X, axis = 1)

    i_k = np.empty((number_paths,M+1))     #+1 weil i_0 mit drin ist
    i_k[:,0] = i_0
    for n in range(number_paths):
        for j in range(M):
            j = j+1
            i_k[n,j] = i_k[n,j-1]*K_0*math.exp(delta_X[n,j-1])
    #i_k = i_k[:,1:]   #i_0 wird abgeschnitten
    #display(i_k)
    return i_k


def create_w_k(i_k, K1, K2, K3, K4, number_paths, M):
    i_k = i_k[:,1:]   #i_0 wird abgeschnitten
    w_k = np.empty((number_paths,M))
    w_k = K1+K2*np.arctan(K3 * i_k + K4)
    return w_k


In [10]:
def create_u_k(number_paths, M, i_k):
    u_k = np.empty((number_paths,M))
    for k in range(M):
        u_k[:,k] = np.prod(1/(i_k[:,0:k+1] + 1), axis = 1)     #hier k+1 weil man bei 0 anfängt
    return u_k

def create_c_k(number_paths, M, i_0):
    c_k = np.ones((number_paths,M))
    for k in range(M):                 
        for j in range(M - (k+1)):
            j = j + 1                   #damit man nicht bei 0 anfängt, erster Summand ist schon 1
            c_k[:,k] = c_k[:,k] + (1 + i_0)**(-j)
    return c_k

def create_r_k(number_paths, M, w_k):
    r_k = np.empty((number_paths, M))
    for k in range(M):
        r_k[:,k] = np.prod(1-w_k[:,0:k] , axis = 1)
    return r_k

def create_m_k(c, w_k, c_k, r_k, number_paths, M):
    m_k = np.empty((number_paths, M))
    for k in range(M):
        m_k[:,k] = c*r_k[:,k]*((1-w_k[:,k]) + w_k[:,k]*c_k[:,k])
    return m_k



# Monte Carlo Simulation

Monte Carlo Simulation mit Standard BM bzw. random walk und Brownian Bridges

## Set parameters

In [13]:
m = 16
number_paths = 2**m #für Sobol sequence
sigma = np.sqrt(0.0004)
M = 360         #Anzahl der simulierten Monate
g = get_BB_power2(M)                   #numer of simulated points excluding the first one
delta_t = 1 #muss glaube ich 1 sein, weil sonst stimmt die Varianz nicht 


K_0 = math.exp(-(sigma**2)/2)
K1 = 0.01
K2 = -0.005
K3 = 10
K4 = 0.5

c = 1 #monthly payment (beliebig?)

i_0 = 0.007

BB = False
QMC = True
Sobol_not_Halton = True
scramble_func = False


In dieser Version des Codes werden zuerst die Folgen kreiert, diese dann entsprechend manipuliert, um damit dann die Simulation durchzuführen und anschließend die varianzen der f_u zu schätzen

In [14]:
#Matrizen mit Transformierten Pseudozufallszahlen, Sobol oder Halton
number_of_run = 1
if QMC == True:
    if Sobol_not_Halton == True:
        norm_rv = Sobol_seq_normal_transformed(M,m, number_paths, scramble_func, number_of_run)
    elif Sobol_not_Halton == False:
        norm_rv = Halton_seq_normal_transformed(M, number_paths, scramble_func, number_of_run)
elif QMC == False:
    norm_rv = np.random.randn(number_paths,M)

In [ ]:
#zuerst generieren wir die Varianz der gesamten Funktion
#Achtung i_0 fehlt in i_k
i_k = create_i_k(BB, number_paths, M, sigma, g, delta_t, m, scramble_func, Sobol_not_Halton, QMC, i_0, K_0, norm_rv)
w_k = create_w_k(i_k, K1, K2, K3, K4, number_paths, M)
u_k = create_u_k(number_paths, M, i_k)
c_k = create_c_k(number_paths, M, i_0)
r_k = create_r_k(number_paths, M, w_k)
m_k = create_m_k(c, w_k, c_k, r_k, number_paths, M)


expectation = np.sum(np.sum(u_k * m_k,axis=1))/number_paths

var_whole_function = (1/(number_paths - 1)) * np.sum((np.sum(u_k * m_k,axis=1) - expectation)**2)  #klasischer Schätzer für varianz
print('total Var: ' + str(var_whole_function))




#while var <0.99 * var_whole_function: #rausgenommen, weil sonst auf jeden Fall diese Schranke erreicht wird
d_T = 1
while d_T<359:
    results_effective_Dimension_Trunkierung = open("effective_Dimension_Trunkierung.txt", "a")
    results_effective_Dimension_Trunkierung.write("d_T "+str(d_T) + "------------------------------------" + "\n")
    results_effective_Dimension_Trunkierung.close()
    var = 0
    for r in range(d_T):
        r = r+1
        for u in itertools.combinations(range(d_T), r): #wir von 0 bis 360
            print(u)
            #Matrizen mit Transformierten Pseudozufallszahlen, Sobol oder Halton
            #ACHTUNG: ich bin mir nicht sicher, ob hier die Sobol, Halton Folge nicht non-overlapping sein müsste, damit auch jedesmal neue Pseudozufallszahlen generiert werden wird jedes Mal ein neuer norm_rv erzeugt
            if QMC == True:

                #die folgende Zeile habe ich wieder rausgenommen für einen besseren Vergleich, evtl. sogar seed für PRNG setzen?
                #number_of_run += 1 #number_of_run wird hochgezählt, damit beim nächsten Durchgang eine nicht-überlappende Teilfolge benutzt wird

                if Sobol_not_Halton == True:
                    norm_rv = Sobol_seq_normal_transformed(M,m, number_paths, scramble_func, number_of_run)
                elif Sobol_not_Halton == False:
                    norm_rv = Halton_seq_normal_transformed(M, number_paths, scramble_func, number_of_run)
            elif QMC == False:
                norm_rv = np.random.randn(number_paths,M)
            norm_rv_edited = norm_rv
            for i in range(M):
                if i not in u:
                    norm_rv_edited[:,i] = 0 #wir setzen alle Werte, die nicht in u sind auf 0 (Normalverteilung)

            i_k = create_i_k(BB, number_paths, M, sigma, g, delta_t, m, scramble_func, Sobol_not_Halton, QMC, i_0, K_0, norm_rv_edited)
            w_k = create_w_k(i_k, K1, K2, K3, K4, number_paths, M)
            u_k = create_u_k(number_paths, M, i_k)
            c_k = create_c_k(number_paths, M, i_0)
            r_k = create_r_k(number_paths, M, w_k)
            m_k = create_m_k(c, w_k, c_k, r_k, number_paths, M)


            expectation = np.sum(np.sum(u_k * m_k,axis=1))/number_paths

            additional_var = (1/(number_paths - 1)) * np.sum((np.sum(u_k * m_k,axis=1) - expectation)**2)  #klasischer Schätzer für varianz
            var = var + additional_var

            results_effective_Dimension_Trunkierung = open("effective_Dimension_Trunkierung.txt", "a")
            results_effective_Dimension_Trunkierung.write(" r "+str(r) + "u " +str(u) +  " additional_var "+str(additional_var) + " var " + str(var) +   "\n")
            results_effective_Dimension_Trunkierung.close()
    d_T += 1
#print("r" + str(r))
#print(var)

